In [ ]:
import numpy as np
import faiss
import litellm
from litellm import completion
from dotenv import load_dotenv
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Load API keys
load_dotenv(".env")
apikey = os.getenv('OPENAI_API_KEY')
print(apikey)

In [ ]:
import requests

# Example query to the arXiv API directly
url = "http://export.arxiv.org/api/query"
params = {
    "search_query": "all:Retrieval Augmented Generation",
    "start": 0,
    "max_results": 5
}

response = requests.get(url, params=params)

if response.status_code == 200:
    print("API Request Successful")
    print("Response:", response.text[:500])  # Print first 500 characters of the response
else:
    print("Failed to fetch data from arXiv:", response.status_code)

In [ ]:
# Load the document pertaining to a particular topic
docs = ArxivLoader(query=""" all:"attention mechanisms" AND (all:"convolutional neural networks" OR all:"CNN") AND NOT all:"transformer" """, load_max_docs=5).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)
print(chunked_documents)

In [ ]:
print(type(docs))
print(len(docs))
print(type(docs[0].page_content))

print(type(chunked_documents))
print(len(chunked_documents))
print(chunked_documents[0])




In [ ]:
# Test
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": "What is a cat?","role": "user"}],
    temperature=0.5
)
print(response.choices[0].message.content)

In [ ]:
user_query = input("Ask a research question!")

# Create multiple search queries
search_split_prompt = f"""
Your role is that of a researcher attempting to answer a question. Given a question from the user,
your job is to come up with one of more ArXiv queries that searches for the exact information needed to answer the question.
A single question may need multiple queries. If there are multiple queries, make sure they are separated by "|"

You can include all syntax that involves including multiple terms, search by abstract, title, etc.

Example 1:
Given question: What are the ethical concerns associated with the use of facial recognition technology?
Your Answer: ("facial recognition technology" OR "facial recognition systems" OR "facial recognition software") AND ("ethical concerns" OR "ethical implications" OR "ethical issues")

Example 2:
Given question: What are some prominent attention mechanisms for convolutional neural networks, and how are they used in the autonomous vehicle industry?
Your Answer: all:"attention mechanisms" AND ("convolutional neural networks" OR "CNN") | all:"attention mechanisms" AND ("autonomous vehicles" OR "self-driving cars")

Question: {user_query},

As shown above, your response should be formatted as follows:

ArXiv Query 1 | ArXiv Query 2 | ArXiv Query 3 | ...

"""
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt4-1106-preview",
    custom_llm_provider="openai",
    messages = [{ "content": search_split_prompt,"role": "user"}],
    temperature=0.5
)
print("Response recieved")
print(response.choices[0].message.content)
arxiv_queries_list = response.choices[0].message.content.split("|")


# Each element contains vector stores for each search query developed by LLM
chunks_for_queries = []
for q in arxiv_queries_list:
    docs = ArxivLoader(query=q, load_max_docs=5).load()
    print(docs)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=350, chunk_overlap=50
    )
    chunked_documents = text_splitter.split_documents(docs)
    chunks_for_queries.append(chunked_documents)

print(type(chunks_for_queries[0][0].page_content))
print(chunks_for_queries[0][1].page_content)

In [ ]:
print(type(chunks_for_queries))
print(len(chunks_for_queries[0]))

In [ ]:
# Instantiate the Embedding Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=apikey, base_url="https://drchat.xyz")
# Create Index- Load document chunks into the vectorstore
vectorstore_list = []
for x in chunks_for_queries:
    faiss_vectorstore = FAISS.from_documents(
        documents=x,
        embedding=embeddings,
    )
    print(type(faiss_vectorstore))
    vectorstore_list.append(faiss_vectorstore)


# Create a retriver and retrieve relevant documents for each vector store
relevant_documents_list = []
for x in vectorstore_list:
    
    relevant_documents = x.similarity_search(user_query, k = 5)
    print(type(relevant_documents))
    relevant_documents_list.append(relevant_documents)

In [ ]:
print(relevant_documents_list)

In [ ]:
question_prompt = f"""
Given the following context: {relevant_documents_list}

Answer the following question: {user_query}

Only answer the question if the answer is in the context. Otherwise, say that you don't know.
"""

response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": question_prompt,"role": "user"}],
    temperature=0.5
)

print(response.choices[0].message.content)